<a href="https://colab.research.google.com/github/alfcastillo90/estructurando_proyecto_data_science_iii/blob/main/Estructurando_proyecto_de_ciencia_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Instalaciones requeridas

In [2]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install mlxtend

# Lectura del dataset

In [11]:
import pandas as pd
path = "/content/gdrive/MyDrive/Estudios/coderhouse/data_science/melb_data.csv";
dataset = pd.read_csv(path, sep=",")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [12]:
# Nombres de las columnas del DataFrame
print("\nNombres de las columnas:")
dataset.columns


Nombres de las columnas:


Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [13]:
#eliminamos los NaN
dataset = dataset.dropna()

# Implementación de metodo de eliminación hacia atrás

In [14]:
# Supongamos que 'Price' es la columna objetivo en tu conjunto de datos
# y que quieres eliminarla de la matriz de características y guardarla en una variable separada.

# Crea la matriz de características eliminando la columna 'Price'
X = dataset.drop("Price", axis=1)

# Crea la característica objetivo guardando la columna 'Price' en una variable separada
y = dataset['Price']

# Muestra las primeras 5 filas del DataFrame original
dataset.head()


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0


In [15]:
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib

In [21]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
import numpy as np

#creamos un nuevo dataset nummerico
numeric_dataset = dataset.select_dtypes(include=[np.number])


sfs = SFS(LinearRegression(),
          k_features=11,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

sfs = sfs.fit(numeric_dataset.drop("Price", axis=1), numeric_dataset["Price"])
sfs.k_feature_names_


('Rooms',
 'Distance',
 'Postcode',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount')

In [22]:
import statsmodels.api as sm

def backward_elimination(data, target, significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

# Aplicamos la funcion solo a cambpos numericos
selected_features = backward_elimination(numeric_dataset.drop("Price", axis=1), numeric_dataset["Price"])

print("Features elegidas ", selected_features)


The selected features are:  ['Rooms', 'Distance', 'Postcode', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']


# Algoritmo de regresión para entrenar los datos

In [23]:
# Importar las bibliotecas necesarias
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X = numeric_dataset[selected_features]
y = numeric_dataset['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones usando el conjunto de prueba
y_pred = model.predict(X_test)

# Imprimir los coeficientes del modelo
print('Coeficientes: \n', model.coef_)

# Imprimir el error cuadrático medio
print('Error cuadrático medio: %.2f'
      % mean_squared_error(y_test, y_pred))

# Imprimir el coeficiente de determinación: 1 es una predicción perfecta
print('Coeficiente de determinación: %.2f'
      % r2_score(y_test, y_pred))


Coeficientes: 
 [ 1.99166109e+05 -4.11029364e+04  8.86500480e+02  2.10740575e+05
  7.18279132e+04  1.77797724e+01  1.68571126e+03 -4.25318769e+03
 -1.10417301e+06  7.50119417e+05 -2.83738116e+00]
Error cuadrático medio: 153245438333.05
Coeficiente de determinación: 0.63


#calculo de metricas para validar el modelo

In [24]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Error cuadrático medio (MSE): {mse}")

# Calcular la raíz del error cuadrático medio (RMSE)
rmse = np.sqrt(mse)
print(f"Raíz del error cuadrático medio (RMSE): {rmse}")

# Calcular el error absoluto medio (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Error absoluto medio (MAE): {mae}")

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print(f"Coeficiente de determinación (R^2): {r2}")


Error cuadrático medio (MSE): 153245438333.05005
Raíz del error cuadrático medio (RMSE): 391465.75627128617
Error absoluto medio (MAE): 270386.8347454586
Coeficiente de determinación (R^2): 0.6313130940072882
